#**Feature Engineering Exercise**

Data processing and feature
engineering is often described to be the toughest task or step in building any Machine Learning system by
data scientists. With the need of both domain knowledge as well as mathematical transformations, feature
engineering is often said to be both an art as well as a science. The obvious complexities involve dealing
with diverse types of data and variables. Besides this, each Machine Learning problem or task needs
specific features and there is no one solution fits all in the case of feature engineering. This makes feature
engineering all the more difficult and complex.

Adapted from Dipanjan Sarkar et al. 2018. [Practical Machine Learning with Python](https://link.springer.com/book/10.1007/978-1-4842-3207-1).

# Feature Engineering on Numeric Data



Even though numeric data can be directly fed into Machine Learning models, you would still need to
engineer features that are relevant to the scenario, problem, and domain before building a model. Hence
the need for feature engineering remains. Important aspects of numeric features include feature scale and
distribution. In some scenarios,
we need to apply specific transformations to change the scale of numeric values and in other scenarios we
need to change the overall distribution of the numeric values, like transforming a skewed distribution to a
normal distribution.

In [ ]:
# Import necessary dependencies and settings
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.stats as spstats

%matplotlib inline
mpl.style.reload_library()
mpl.style.use('classic')
mpl.rcParams['figure.facecolor'] = (1, 1, 1, 0)
mpl.rcParams['figure.figsize'] = [6.0, 4.0]
mpl.rcParams['figure.dpi'] = 100

## Raw Measures

Raw measures typically
indicated using numeric variables directly as features without any form of transformation or engineering.
Typically these features can indicate values or counts.

###Values

Usually, scalar values in its raw form indicate a specific measurement, metric, or observation belonging to
a specific variable or field. The semantics of this field is usually obtained from the field name itself or a data
dictionary if present.

###Ecoli Dataset

Ecoli dataset is for predicting Protein Localization Sites in Ecoli. It includes 336 samples and 8 attributes (7 predictive and 1 target of localization site).

You can learn more about the dataset here:
* Ecoli Dataset ([ecoli.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ecoli.data))
* Ecoli Dataset Description ([ecoli.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ecoli.names))

Attribute Information.

1.  **accession**: Accession number for the SWISS-PROT database
1.  **mcg**: McGeoch's method for signal sequence recognition.
1.  **gvh**: von Heijne's method for signal sequence recognition.
1.  **lip**: von Heijne's Signal Peptidase II consensus sequence score (Binary attribute).
1.  **chg**: Presence of charge on N-terminus of predicted lipoproteins. Binary attribute.
1.  **aac**: score of discriminant analysis of the amino acid content of outer membrane and periplasmic proteins.
1. **alm1**: score of the ALOM membrane spanning region prediction program.
1. **alm2**: score of ALOM program after excluding putative cleavable signal regions from the sequence.


The localization site. 

```
  cp  | cytoplasm                                   | 143
  im  | inner membrane without signal sequence      |  77 
  pp  | perisplasm                                  |  52
  imU | inner membrane, uncleavable signal sequence |  35
  om  | outer membrane                              |  20
  omL | outer membrane lipoprotein                  |   5
  imL | inner membrane lipoprotein                  |   2
  imS | inner membrane, cleavable signal sequence   |   2
```

In [ ]:
# Download Ecoli dataset
!wget -O ecoli.csv "https://raw.githubusercontent.com/udel-cbcb/al_ml_workshop/main/data/ecoli.csv"
!head ecoli.csv 

ecoli_df = pd.read_csv('ecoli.csv')
ecoli_df.head(10)

--2022-05-20 03:07:36--  https://raw.githubusercontent.com/udel-cbcb/al_ml_workshop/main/data/ecoli.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16498 (16K) [text/plain]
Saving to: ‘ecoli.csv’

ecoli.csv           100%[===================>]  16.11K  --.-KB/s    in 0s      

2022-05-20 03:07:36 (90.9 MB/s) - ‘ecoli.csv’ saved [16498/16498]

accession,mcg,gvh,lip,chg,aac,alm1,alm2,site
EMRA_ECOLI,0.06,0.61,0.48,0.50,0.49,0.92,0.37,im
AAT_ECOLI,0.49,0.29,0.48,0.50,0.56,0.24,0.35,cp
ATKC_ECOLI,0.85,0.53,0.48,0.50,0.53,0.52,0.35,imS
ACEA_ECOLI,0.07,0.40,0.48,0.50,0.54,0.35,0.44,cp
FADL_ECOLI,0.78,0.68,0.48,0.50,0.83,0.40,0.29,om
NLPA_ECOLI,0.75,0.55,1.00,1.00,0.40,0.47,0.30,imL
MULI_ECOLI,0.77,0.57,1.00,0.50,0.37,0.54,0.01,omL
ACEK_ECOLI,0.56,0.40,0.48,0.5

,accession,mcg,gvh,lip,chg,aac,alm1,alm2,site
0,EMRA_ECOLI,0.06,0.61,0.48,0.5,0.49,0.92,0.37,im
1,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp
2,ATKC_ECOLI,0.85,0.53,0.48,0.5,0.53,0.52,0.35,imS
3,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp
4,FADL_ECOLI,0.78,0.68,0.48,0.5,0.83,0.40,0.29,om
5,NLPA_ECOLI,0.75,0.55,1.00,1.0,0.40,0.47,0.30,imL
6,MULI_ECOLI,0.77,0.57,1.00,0.5,0.37,0.54,0.01,omL
7,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp
8,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp
9,AGP_ECOLI,0.74,0.49,0.48,0.5,0.42,0.54,0.36,pp


In [ ]:
# Show some of features
ecoli_df[['mcg', 'gvh', 'chg']].head()

,mcg,gvh,chg
0,0.06,0.61,0.5
1,0.49,0.29,0.5
2,0.85,0.53,0.5
3,0.07,0.40,0.5
4,0.78,0.68,0.5


In [ ]:
# Compute basic statistical measures on the fields of 'mcg', 'gvh', 'chg'
# Your code goes here

,mcg,gvh,lip,chg,aac,alm1,alm2,siteLabel
count,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000
mean,0.500060,0.500000,0.495476,0.501488,0.500030,0.500179,0.499732,2.145833
std,0.194634,0.148157,0.088495,0.027277,0.122376,0.215751,0.209411,2.645105
min,0.000000,0.160000,0.480000,0.500000,0.000000,0.030000,0.000000,0.000000
25%,0.340000,0.400000,0.480000,0.500000,0.420000,0.330000,0.350000,0.000000
50%,0.500000,0.470000,0.480000,0.500000,0.495000,0.455000,0.430000,1.000000
75%,0.662500,0.570000,0.480000,0.500000,0.570000,0.710000,0.710000,4.000000
max,0.890000,1.000000,1.000000,1.000000,0.880000,1.000000,0.990000,7.000000


###Counts
Raw numeric measures can also indicate counts, frequencies and occurrences of specific attributes.

###Diabetes Dataset
The dataset classifies patient data as
either an onset of diabetes within five years or not. There are 768 examples and eight input variables. It is a binary classification problem. 

You can learn more about the dataset here:

* Diabetes Dataset File ([pima-indians-diabetes.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv))
* Diabetes Dataset Details ([pima-indians-diabetes.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names))

In [ ]:
# Download Diabetes dataset
!wget -O pima-indians-diabetes.csv "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"
!head pima-indians-diabetes.csv 

--2022-05-20 03:07:37--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0s      

2022-05-20 03:07:37 (98.4 MB/s) - ‘pima-indians-diabetes.csv’ saved [23278/23278]

6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
5,116,74,0,0,25.6,0.201,30,0
3,78,50,32,88,31.0,0.248,26,1
10,115,0,0,0,35.3,0.134,29,0
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0.0,0.232,54,1


In [ ]:
diabetes_df = pd.read_csv('pima-indians-diabetes.csv', header=None)
diabetes_df.columns=['pregnancy', 'glucose', 'bp', 'triceps', 'insulin', 'bmi', 'pedigree', 'age', 'diabetes']
diabetes_df.head(10)

,pregnancy,glucose,bp,triceps,insulin,bmi,pedigree,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [ ]:
diabetes_df.describe()

,pregnancy,glucose,bp,triceps,insulin,bmi,pedigree,age,diabetes
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


##Binarization

If you are more concerned about the various songs he/she has listened to. In this case, a binary
feature is preferred as opposed to a count based feature.

In [ ]:
# Binarize 'age' field manually
age = np.array(diabetes_df['age']) 
old = np.array(diabetes_df['age']) 
old[age > 50] = 1
old[age <= 50] = 0
diabetes_df['old'] = old

diabetes_df.head(10)

,pregnancy,glucose,bp,triceps,insulin,bmi,pedigree,age,diabetes,old
0,6,148,72,35,0,33.6,0.627,50,1,0
1,1,85,66,29,0,26.6,0.351,31,0,0
2,8,183,64,0,0,23.3,0.672,32,1,0
3,1,89,66,23,94,28.1,0.167,21,0,0
4,0,137,40,35,168,43.1,2.288,33,1,0
5,5,116,74,0,0,25.6,0.201,30,0,0
6,3,78,50,32,88,31.0,0.248,26,1,0
7,10,115,0,0,0,35.3,0.134,29,0,0
8,2,197,70,45,543,30.5,0.158,53,1,1
9,8,125,96,0,0,0.0,0.232,54,1,1


In [ ]:
# Binarize 'age' field using Binarizer
from sklearn.preprocessing import Binarizer

# Binarize data (set feature values to 0 or 1) according to a threshold.
# Values greater than the threshold map to 1, while values less than
# or equal to the threshold map to 0. With the default threshold of 0,
# only positive values map to 1.

bn = # Your code goes here
bn_old = bn.transform([diabetes_df['age']])[0]
diabetes_df['bn_old'] = bn_old
diabetes_df.head(10)

,pregnancy,glucose,bp,triceps,insulin,bmi,pedigree,age,diabetes,old,bn_old
0,6,148,72,35,0,33.6,0.627,50,1,0,0
1,1,85,66,29,0,26.6,0.351,31,0,0,0
2,8,183,64,0,0,23.3,0.672,32,1,0,0
3,1,89,66,23,94,28.1,0.167,21,0,0,0
4,0,137,40,35,168,43.1,2.288,33,1,0,0
5,5,116,74,0,0,25.6,0.201,30,0,0,0
6,3,78,50,32,88,31.0,0.248,26,1,0,0
7,10,115,0,0,0,35.3,0.134,29,0,0,0
8,2,197,70,45,543,30.5,0.158,53,1,1,1
9,8,125,96,0,0,0.0,0.232,54,1,1,1


##Rounding
Often when dealing with numeric attributes like proportions or percentages, we may not need values with a
high amount of precision. Hence it makes sense to round off these high precision percentages into numeric
integers. These integers can then be directly used as raw numeric values or even as categorical (discreteclass
based) features.

In [ ]:
# Creare a column 'pedigree_scale_10' and rounding off the 'pedigree' by 10
diabetes_df['pedigree_scale_10'] = np.array(np.round((diabetes_df['pedigree'] * 10)), dtype='int')
# Creare a column 'popularity_scale_100' and rounding off the 'pop_percent' by 100
diabetes_df['pedigree_scale_100'] = # Your code goes here
diabetes_df

array([ 0,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 56, 60, 63, 99])

##Interactions
Often in real-world datasets and scenarios, it makes sense to also try to capture the
interactions between these feature variables as a part of the input feature set.

In [ ]:
gvh_lip = ecoli_df[['gvh','lip']]
gvh_lip.head()

,gvh,lip
0,0.61,0.48
1,0.29,0.48
2,0.53,0.48
3,0.40,0.48
4,0.68,0.48


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# build features up to the second degree using the PolynomialFeatures class from scikit-learn's API.
pf = # Your code goes here
res = pf.fit_transform(gvh_lip)
res

array([[0.61  , 0.48  , 0.3721, 0.2928, 0.2304],
       [0.29  , 0.48  , 0.0841, 0.1392, 0.2304],
       [0.53  , 0.48  , 0.2809, 0.2544, 0.2304],
       ...,
       [0.6   , 0.48  , 0.36  , 0.288 , 0.2304],
       [0.61  , 0.48  , 0.3721, 0.2928, 0.2304],
       [0.74  , 0.48  , 0.5476, 0.3552, 0.2304]])

We have a total of five features including the new interaction
features.

We can see the degree of each feature in the matrix.

In [2]:
pd.DataFrame(pf.powers_, columns=['gvh_degree', 'lip_degree'])

NameError: ignored

Now that we know what each feature actually represented from the degrees depicted, we can assign a
name to each feature as follows to get the updated feature set.

In [ ]:
intr_features = pd.DataFrame(res, columns=['gvh', 'lip', 'gvh^2', 'gvh x lip', 'lip^2'])
intr_features.head(5)  

,gvh,lip,gvh^2,gvh x lip,lip^2
0,0.61,0.48,0.3721,0.2928,0.2304
1,0.29,0.48,0.0841,0.1392,0.2304
2,0.53,0.48,0.2809,0.2544,0.2304
3,0.40,0.48,0.1600,0.1920,0.2304
4,0.68,0.48,0.4624,0.3264,0.2304


Transforming new data in the future (during predictions)

In [1]:
# take some sample new observations for gvh and lip features and try to transform
# them using this same mechanism.
new_df = pd.DataFrame([[0.35, 0.49],[0.46, 0.38], [0.25, 0.48]], 
                      columns=['gvh', 'lip'])
new_df

NameError: ignored

In [ ]:
# use the pf object that we created earlier and transform these input features to give us the
# interaction features
new_res = pf.transform(new_df)
new_intr_features = pd.DataFrame(new_res, 
                                 columns=['gvh', 'lip', 'gvh^2', 'gvh x lip', 'lip^2'])
new_intr_features

,gvh,lip,gvh^2,gvh x lip,lip^2
0,95.0,75.0,9025.0,7125.0,5625.0
1,121.0,120.0,14641.0,14520.0,14400.0
2,77.0,60.0,5929.0,4620.0,3600.0


#Feature Engineering on Categorical Data

Any attribute or feature that is categorical in nature represents discrete values that belong to a specific
finite set of categories or classes. Category or class labels can be text or numeric in nature. Usually there are
two types of categorical variables—nominal and ordinal.


In [ ]:
# Import necessary dependencies and settings
import pandas as pd
import numpy as np

##Transforming Nominal Features

Nominal features or attributes are categorical variables that usually have a finite set of distinct discrete
values. Often these values are in string or text format and Machine Learning algorithms cannot understand
them directly. Hence usually you might need to transform these features into a more representative numeric
format.

In [ ]:
ecoli_df.head(11)

,accession,mcg,gvh,lip,chg,aac,alm1,alm2,site,siteLabel
0,EMRA_ECOLI,0.06,0.61,0.48,0.5,0.49,0.92,0.37,im,1
1,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp,0
2,ATKC_ECOLI,0.85,0.53,0.48,0.5,0.53,0.52,0.35,imS,3
3,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp,0
4,FADL_ECOLI,0.78,0.68,0.48,0.5,0.83,0.40,0.29,om,5
5,NLPA_ECOLI,0.75,0.55,1.00,1.0,0.40,0.47,0.30,imL,2
6,MULI_ECOLI,0.77,0.57,1.00,0.5,0.37,0.54,0.01,omL,6
7,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp,0
8,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp,0
9,AGP_ECOLI,0.74,0.49,0.48,0.5,0.42,0.54,0.36,pp,7


The dataset depicted in this dataframe shows us various attributes pertaining to video games. Features
like Platform, Genre, and Publisher are nominal categorical variables.

In [ ]:
sites = np.unique(ecoli_df['site'])
sites

array(['cp', 'im', 'imL', 'imS', 'imU', 'om', 'omL', 'pp'], dtype=object)

This output tells us we have 8 distinct sites in Ecoli dataset. 

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Let’s transform this feature now using a mapping scheme of 'site'

sle = # Your code goes here

site_labels = # Your code goes here
site_mappings = {index: label for index, label in enumerate(sle.classes_)}
site_mappings

{0: 'cp', 1: 'im', 2: 'imL', 3: 'imS', 4: 'imU', 5: 'om', 6: 'omL', 7: 'pp'}

A mapping scheme has been generated where each site value is
mapped to a number with the help of the LabelEncoder object sle. The transformed labels are stored in the
site_labels value.

In [ ]:
ecoli_df['siteLabel'] = # Your code goes here
ecoli_df.head(11)

,accession,mcg,gvh,lip,chg,aac,alm1,alm2,site,siteLabel
0,EMRA_ECOLI,0.06,0.61,0.48,0.5,0.49,0.92,0.37,im,1
1,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp,0
2,ATKC_ECOLI,0.85,0.53,0.48,0.5,0.53,0.52,0.35,imS,3
3,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp,0
4,FADL_ECOLI,0.78,0.68,0.48,0.5,0.83,0.40,0.29,om,5
5,NLPA_ECOLI,0.75,0.55,1.00,1.0,0.40,0.47,0.30,imL,2
6,MULI_ECOLI,0.77,0.57,1.00,0.5,0.37,0.54,0.01,omL,6
7,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp,0
8,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp,0
9,AGP_ECOLI,0.74,0.49,0.48,0.5,0.42,0.54,0.36,pp,7


The SiteLabel field shows the mapped numeric labels for each of the site labels and we can clearly
see that this adheres to the mappings that we generated earlier.

##Transforming Ordinal Features

Ordinal features are similar to nominal features except that order matters and is an inherent property with
which we can interpret the values of these features. Like nominal features, even ordinal features might be
present in text form and you need to map and transform them into their numeric representation.

In [ ]:
# Pokemon dataset is fictional animals also available on Kaggle.
!wget -O Pokemon.csv "https://raw.githubusercontent.com/dipanjanS/practical-machine-learning-with-python/master/notebooks/Ch04_Feature_Engineering_and_Selection/datasets/Pokemon.csv"
!head Pokemon.csv 

poke_df = pd.read_csv('Pokemon.csv', encoding='utf-8')
poke_df.head()

--2022-05-20 04:19:58--  https://raw.githubusercontent.com/dipanjanS/practical-machine-learning-with-python/master/notebooks/Ch04_Feature_Engineering_and_Selection/datasets/Pokemon.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47228 (46K) [text/plain]
Saving to: ‘Pokemon.csv’

Pokemon.csv         100%[===================>]  46.12K  --.-KB/s    in 0.004s  

2022-05-20 04:19:59 (12.4 MB/s) - ‘Pokemon.csv’ saved [47228/47228]

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,Gen 1,FALSE
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,Gen 1,FALSE
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,Gen 1,FALSE
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,Gen 1,F

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,Gen 1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,Gen 1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,Gen 1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,Gen 1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,Gen 1,False


In [ ]:
poke_df = pd.read_csv('Pokemon.csv', encoding='utf-8')
poke_df = poke_df.sample(random_state=1, frac=1).reset_index(drop=True)

np.unique(poke_df['Generation'])

array(['Gen 1', 'Gen 2', 'Gen 3', 'Gen 4', 'Gen 5', 'Gen 6'], dtype=object)

From this output we can see that there are a total of six
generations of Pok mon. This attribute is definitely ordinal because Pok mon belonging to Generation 1
were introduced earlier in the video games and the television shows than Generation 2 and so on. Hence
they have a sense of order among them. 

However, there is no generic module or function to map and transform these features
into numeric representations. Hence we need to hand-craft this using our own logic, which is depicted in the
following code snippet.

In [ ]:
gen_ord_map = {'Gen 1': 1, 'Gen 2': 2, 'Gen 3': 3, 
               'Gen 4': 4, 'Gen 5': 5, 'Gen 6': 6}

poke_df['GenerationLabel'] = # Your code goes here
poke_df[['Name', 'Generation', 'GenerationLabel']].iloc[4:10]

,Name,Generation,GenerationLabel
4,Octillery,Gen 2,2
5,Helioptile,Gen 6,6
6,Dialga,Gen 4,4
7,DeoxysDefense Forme,Gen 3,3
8,Rapidash,Gen 1,1
9,Swanna,Gen 5,5


##Encoding Categorical Features

If we directly fed these transformed numeric
representations of categorical features into any algorithm, the model will essentially try to interpret these as
raw numeric features and hence the notion of magnitude will be wrongly introduced in the system.

Hence models built using these features directly would
be sub-optimal and incorrect models. There are several schemes and strategies where dummy features are
created for each unique value or label out of all the distinct categories in any feature. In the subsequent
sections, we will discuss some of these schemes including one hot encoding, dummy coding, effect coding,
and feature hashing schemes.

###One Hot Encoding Scheme
Considering we have numeric representation of any categorical feature with m labels, the one hot encoding
scheme, encodes or transforms the feature into m binary features, which can only contain a value of 1 or 0. Each observation in the categorical feature is thus converted into a vector of size m with only one of the
values as 1 (indicating it as active).

In [ ]:
poke_df[['Name', 'Generation', 'Legendary']].iloc[4:10]

,Name,Generation,Legendary
4,Octillery,Gen 2,False
5,Helioptile,Gen 6,False
6,Dialga,Gen 4,True
7,DeoxysDefense Forme,Gen 3,True
8,Rapidash,Gen 1,False
9,Swanna,Gen 5,False


In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# transform and map pokemon generations
gen_le = LabelEncoder()
gen_labels = gen_le.fit_transform(poke_df['Generation'])
poke_df['Gen_Label'] = gen_labels

# transform and map pokemon legendary status
leg_le = # Your code goes here
leg_labels = # Your code goes here
poke_df['Lgnd_Label'] = # Your code goes here

poke_df_sub = poke_df[['Name', 'Generation', 'Gen_Label', 'Legendary', 'Lgnd_Label']]
poke_df_sub.iloc[4:10]

,Name,Generation,Gen_Label,Legendary,Lgnd_Label
4,Octillery,Gen 2,1,False,0
5,Helioptile,Gen 6,5,False,0
6,Dialga,Gen 4,3,True,1
7,DeoxysDefense Forme,Gen 3,2,True,1
8,Rapidash,Gen 1,0,False,0
9,Swanna,Gen 5,4,False,0


In [ ]:
# encode generation labels using one-hot encoding scheme
gen_ohe = OneHotEncoder()
gen_feature_arr = gen_ohe.fit_transform(poke_df[['Gen_Label']]).toarray()
gen_feature_labels = list(gen_le.classes_)
gen_features = pd.DataFrame(gen_feature_arr, columns=gen_feature_labels)

# encode legendary status labels using one-hot encoding scheme
leg_ohe = # Your code goes here
leg_feature_arr = # Your code goes here
leg_feature_labels = ['Legendary_'+str(cls_label) for cls_label in leg_le.classes_]
leg_features = # Your code goes here

In [ ]:
# Let’s now concatenate these feature frames and see the final result.
poke_df_ohe = pd.concat([poke_df_sub, gen_features, leg_features], axis=1)
columns = sum([['Name', 'Generation', 'Gen_Label'],gen_feature_labels,
              ['Legendary', 'Lgnd_Label'],leg_feature_labels], [])
poke_df_ohe[columns].iloc[4:10]

,Name,Generation,Gen_Label,Gen 1,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6,Legendary,Lgnd_Label,Legendary_False,Legendary_True
4,Octillery,Gen 2,1,0.0,1.0,0.0,0.0,0.0,0.0,False,0,1.0,0.0
5,Helioptile,Gen 6,5,0.0,0.0,0.0,0.0,0.0,1.0,False,0,1.0,0.0
6,Dialga,Gen 4,3,0.0,0.0,0.0,1.0,0.0,0.0,True,1,0.0,1.0
7,DeoxysDefense Forme,Gen 3,2,0.0,0.0,1.0,0.0,0.0,0.0,True,1,0.0,1.0
8,Rapidash,Gen 1,0,1.0,0.0,0.0,0.0,0.0,0.0,False,0,1.0,0.0
9,Swanna,Gen 5,4,0.0,0.0,0.0,0.0,1.0,0.0,False,0,1.0,0.0


We can clearly see the new one hot encoded features
for Gen_Label and Lgnd_Label. Each of these one hot encoded features is binary in nature and if they
contain the value 1, it means that feature is active for the corresponding observation.

In [ ]:
# The following code shows us two dummy data points pertaining to new Pokemon.
new_poke_df = pd.DataFrame([['PikaZoom', 'Gen 3', True], 
                           ['CharMyToast', 'Gen 4', False]],
                           columns=['Name', 'Generation', 'Legendary'])
new_poke_df

,Name,Generation,Legendary
0,PikaZoom,Gen 3,True
1,CharMyToast,Gen 4,False


In [ ]:
# converting the text categories into numeric representations using our previously built LabelEncoder objects
new_gen_labels = gen_le.transform(new_poke_df['Generation'])
new_poke_df['Gen_Label'] = new_gen_labels

new_leg_labels = leg_le.transform(new_poke_df['Legendary'])
new_poke_df['Lgnd_Label'] = new_leg_labels

new_poke_df[['Name', 'Generation', 'Gen_Label', 'Legendary', 'Lgnd_Label']]

,Name,Generation,Gen_Label,Legendary,Lgnd_Label
0,PikaZoom,Gen 3,2,True,1
1,CharMyToast,Gen 4,3,False,0


In [ ]:
# use our previously built LabelEncoder objects and perform one hot encoding on these new data observations
new_gen_feature_arr = gen_ohe.transform(new_poke_df[['Gen_Label']]).toarray()
new_gen_features = pd.DataFrame(new_gen_feature_arr, columns=gen_feature_labels)

new_leg_feature_arr = # Your code goes here
new_leg_features = # Your code goes here

new_poke_ohe = pd.concat([new_poke_df, new_gen_features, new_leg_features], axis=1)
columns = sum([['Name', 'Generation', 'Gen_Label'], gen_feature_labels,
               ['Legendary', 'Lgnd_Label'], leg_feature_labels], [])
new_poke_ohe[columns]

,Name,Generation,Gen_Label,Gen 1,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6,Legendary,Lgnd_Label,Legendary_False,Legendary_True
0,PikaZoom,Gen 3,2,0.0,0.0,1.0,0.0,0.0,0.0,True,1,0.0,1.0
1,CharMyToast,Gen 4,3,0.0,0.0,0.0,1.0,0.0,0.0,False,0,1.0,0.0


In [ ]:
# Pandas provides to_dummies() function that can help us easily perform one hot encoding
gen_onehot_features = pd.get_dummies(poke_df['Generation'])
pd.concat([poke_df[['Name', 'Generation']], gen_onehot_features], axis=1).iloc[4:10]

,Name,Generation,Gen 1,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6
4,Octillery,Gen 2,0,1,0,0,0,0
5,Helioptile,Gen 6,0,0,0,0,0,1
6,Dialga,Gen 4,0,0,0,1,0,0
7,DeoxysDefense Forme,Gen 3,0,0,1,0,0,0
8,Rapidash,Gen 1,1,0,0,0,0,0
9,Swanna,Gen 5,0,0,0,0,1,0


###Dummy Coding Scheme
The dummy coding scheme is similar to the one hot encoding scheme, except in the case of dummy coding
scheme, when applied on a categorical feature with m distinct labels, we get m-1 binary features. Thus each
value of the categorical variable gets converted into a vector of size m-1. The extra feature is completely
disregarded and thus if the category values range from {0, 1, ..., m-1} the 0th or the m-1th feature is usually
represented by a vector of all zeros (0).

In [ ]:
# Create dummy coding scheme on Pok mon Generation by dropping the first level binary encoded feature (Gen 1).
gen_dummy_features = pd.get_dummies(poke_df['Generation'], drop_first=True)
pd.concat([poke_df[['Name', 'Generation']], gen_dummy_features], axis=1).iloc[4:10]

,Name,Generation,Gen 2,Gen 3,Gen 4,Gen 5,Gen 6
4,Octillery,Gen 2,1,0,0,0,0
5,Helioptile,Gen 6,0,0,0,0,1
6,Dialga,Gen 4,0,0,1,0,0
7,DeoxysDefense Forme,Gen 3,0,1,0,0,0
8,Rapidash,Gen 1,0,0,0,0,0
9,Swanna,Gen 5,0,0,0,1,0


In [ ]:
# choose to drop the last level binary encoded feature (Gen 6)
gen_onehot_features = # Your code goes here
gen_dummy_features = # Your code goes here
pd.concat([poke_df[['Name', 'Generation']], gen_dummy_features], axis=1).iloc[4:10]

,Name,Generation,Gen 1,Gen 2,Gen 3,Gen 4,Gen 5
4,Octillery,Gen 2,0,1,0,0,0
5,Helioptile,Gen 6,0,0,0,0,0
6,Dialga,Gen 4,0,0,0,1,0
7,DeoxysDefense Forme,Gen 3,0,0,1,0,0
8,Rapidash,Gen 1,1,0,0,0,0
9,Swanna,Gen 5,0,0,0,0,1
